In [1]:
from Bio import SeqIO
import os
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk


def read_pssm_file(file_path):
    pssm = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines[3:]:
#             print(line.split(' '))
            line_list = line.split(' ')
            line_list = [item for item in line_list if len(item)!=0]
            if len(line_list[2:22]) == 0:
                break
            pssm.extend(list(map(int,line_list[2:22])))
    return pssm

def getFasta(fasta_path):

    fasta_file = fasta_path

    sequences = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append({
            "id": record.id,
            "sequence": str(record.seq)
        })

    result = []
    
    for sequence in sequences:

        
        if sequence["sequence"].find('O') != -1:
            print(fasta_path)
            continue
        if len(sequence["sequence"])>512:
            continue

        result.append(sequence["sequence"])


    return result




def getPssm(dim):
    folder_path = r"./PSSMs"

    files = os.listdir(folder_path)
    pssm_labels = []
    pssm_features = []
    temp_label = 0
    queDict = dict()
    for file in files:
        sub_files = os.listdir(folder_path + '/' + file)
        print(len(pssm_features))
        if file == 'NMBP':
            temp_label = 0
        else:
#             continue
            temp_label = 1
        queDict[file] = []
        print(temp_label,file)
        for sub_file in sub_files:
            end_file_path = folder_path + '/' + file + '/' + sub_file
            temp = read_pssm_file(end_file_path)
            if len(temp) > 10240:
                continue
            pssm_features.append(temp)
            pssm_labels.append(temp_label)
            queDict[file].append(sub_file.split('.')[0])
#             print(queDict)
    
    new_features = []
    for i in range(len(pssm_features)):
        pssm_features[i].extend([0 for j in range(10240-len(pssm_features[i]))])
        new_features.append(pssm_features[i])
#         print(new_features)
#         new_features.append(pssm_features[i].extend([0]*(10240-len(features[i]))))


    n_components =dim

    pca = PCA(n_components=n_components)

    X_pca = pca.fit_transform(new_features)
    
    return X_pca,queDict



def getData(que_dict):
    folder_path = r'./Reads'

    label = []
    feature = []

    folders = os.listdir(folder_path)
    cnt = 0
    flag = 0
    for folder in folders:
        filds = os.listdir(folder_path + '/' + folder)
        if folder == 'NMBP':
            flag = 0
        else:
            flag = 1
        for fild in filds:
            if fild.split('.')[0] not in que_dict[folder]:

                continue
            false_feature = getFasta(folder_path + '/' + folder + '/' + fild)
            
            false_feature = ' '.join(false_feature[0])
            feature.append(false_feature)
            label.append(flag)
    
    return feature,label
    
    
import os
from sklearn.decomposition import PCA
import numpy as np
import torch
def mix_data(encoded_data,X_pca):
    X_pca = cp.deepcopy(X_pca)
    input_ids = []
    attention_mask = []
    for i in range(len(X_pca)):
        list_encoded = encoded_data['input_ids'][i].tolist()
        list_X_pca = list(X_pca[i])
        list_X_pca.extend(list_encoded)
        input_ids.append(list_X_pca)
        attention_mask_list = encoded_data['attention_mask'][i].tolist()
        attention_mask_listtemp = [1 for i in range(1024-514)]
        attention_mask_listtemp.extend(attention_mask_list)
        attention_mask.append(attention_mask_listtemp)
    
    return input_ids,attention_mask

In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import copy as cp
X_pca,queDict = getPssm(512)
featrue,label = getData(queDict)

0
1 Ca
2950
1 Cu
3358
1 Fe
4886
1 K
5602
1 Mn
6632
0 NMBP
10730
1 Zn


In [3]:
before_featrue = []
for i in featrue:
    before_featrue.append(i.split(' '))

In [4]:
model = Word2Vec(before_featrue, vector_size=50)
word_vectors = model.wv

In [5]:
after_feture = []
maxmax = -1
for i in range(len(before_featrue)):
    temp = []
    for j in range(len(before_featrue[i])):
        temp.extend(word_vectors[before_featrue[i][j]])
    maxmax = max(maxmax,len(temp))
    temp.extend([0 for item in range(25600 - len(temp))])
    after_feture.append(temp)

In [ ]:
pca = PCA(n_components=512)

after_feture_pca = pca.fit_transform(after_feture)

In [ ]:
after_feture_pca = after_feture_pca.tolist()

In [ ]:
all_total_feture = []
for i in range(len(X_pca)):
    after_feture_pca_copy = cp.deepcopy(after_feture_pca[i])
    after_feture_pca_copy.extend(X_pca[i])
    all_total_feture.append(after_feture_pca_copy)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

all_total_feture_normalized = scaler.fit_transform(all_total_feture)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_total_feture_normalized, label, test_size=0.1, random_state=42)

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

model_svm = svm.SVC()

model_svm.fit(X_train, y_train)

y_pred_svm = model_svm.predict(X_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("ACC:", accuracy_svm)


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score




clf_DecisionTreeClassifier = DecisionTreeClassifier(random_state=42)

clf_DecisionTreeClassifier.fit(X_train, y_train)

y_pred_DecisionTreeClassifier = clf_DecisionTreeClassifier.predict(X_test)

accuracy_DecisionTreeClassifier = accuracy_score(y_test, y_pred_DecisionTreeClassifier)
print("ACC：", accuracy_DecisionTreeClassifier)

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


model_RandomForestClassifier = RandomForestClassifier(random_state=42)

model_RandomForestClassifier.fit(X_train, y_train)

y_pred_RandomForestClassifier = model_RandomForestClassifier.predict(X_test)

accuracy_RandomForestClassifier = accuracy_score(y_test, y_pred_RandomForestClassifier)
print("ACC：", accuracy_RandomForestClassifier)


In [ ]:
from catboost import CatBoostClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


model_catboost = CatBoostClassifier(iterations=2500, depth=7, learning_rate=0.06, loss_function='Logloss')

model_catboost.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)

y_pred_prob_catboost = model_catboost.predict(X_test, prediction_type='Probability')[:, 1]  
y_pred_catboost = [1 if pred > 0.5 else 0 for pred in y_pred_prob_catboost] 

accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
print(f'ACC: {accuracy_catboost}')


In [ ]:

import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    'objective': 'binary',
    'metric': 'binary_error', 
    'boosting_type': 'gbdt',
#     'num_leaves': 31,
    'learning_rate': 0.06,
    'feature_fraction': 0.9
}

num_round = 5000
bst_lightgbm = lgb.train(params, train_data, num_round, valid_sets=[test_data],verbose_eval=100)

y_pred_prob_lightgbm = bst_lightgbm.predict(X_test, num_iteration=bst_lightgbm.best_iteration)
y_pred_lightgbm = [1 if pred > 0.5 else 0 for pred in y_pred_prob_lightgbm] 

accuracy_lightgbm = accuracy_score(y_test, y_pred_lightgbm)
print(f'Accuracy: {accuracy_lightgbm}')


In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'binary:logistic',
    'learning_rate': 0.03
}

watchlist = [(dtrain, 'train'), (dtest, 'test')]

num_round = 3000
bst_xgboost = xgb.train(params, dtrain, num_round, evals=watchlist, verbose_eval=100)

dtest = xgb.DMatrix(X_test, label=y_test)
y_pred_prob_xgboost = bst_xgboost.predict(dtest)  
y_pred_xgboost = [1 if pred > 0.5 else 0 for pred in y_pred_prob_xgboost]  
accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)
print(f'Accuracy: {accuracy_xgboost}')


In [ ]:
class_feture_train = []
for i in range(len(y_pred_prob_xgboost)):
    
    class_feture_train.append([y_pred_prob_xgboost[i] , y_pred_prob_lightgbm[i] , y_pred_prob_catboost[i]])

In [ ]:
# class_feture_train

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


y_class_feture_train = cp.deepcopy(y_test)

X_train_vote, X_test_vote, y_train_vote, y_test_vote = train_test_split(class_feture_train, y_class_feture_train, test_size=0.3, random_state=42)


clf = svm.SVC()  


clf.fit(X_train_vote, y_train_vote)

y_pred_vote = clf.predict(X_test_vote)

accuracy_vote = accuracy_score(y_test_vote, y_pred_vote)
print("ACC:", accuracy_vote)

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
colors = sns.color_palette("colorblind", 7)



fpr1, tpr1, thresholds1 = roc_curve(y_test_vote, y_pred_vote)


roc_auc1 = auc(fpr1, tpr1)


fpr2, tpr2, thresholds2 = roc_curve(y_test, y_pred_svm.tolist())


roc_auc2 = auc(fpr2, tpr2)


fpr3, tpr3, thresholds3 = roc_curve(y_test, y_pred_DecisionTreeClassifier)

roc_auc3 = auc(fpr3, tpr3)


fpr4, tpr4, thresholds4 = roc_curve(y_test, y_pred_RandomForestClassifier)

roc_auc4 = auc(fpr4, tpr4)


fpr5, tpr5, thresholds5 = roc_curve(y_test, y_pred_xgboost)

roc_auc5 = auc(fpr5, tpr5)


fpr6, tpr6, thresholds6 = roc_curve(y_test, y_pred_lightgbm)

roc_auc6 = auc(fpr6, tpr6)

fpr7, tpr7, thresholds7 = roc_curve(y_test, y_pred_catboost)

roc_auc7 = auc(fpr7, tpr7)

plt.rcParams['font.size'] = 15  

# 绘制ROC曲线
plt.figure(figsize=(10, 8))
plt.plot(fpr1, tpr1, lw=2, label=f'XLC-S-MIBP ROC curve (area = {roc_auc1:.2f})', color=colors[0])
plt.plot(fpr2, tpr2, lw=2, label=f'SVM ROC curve (area = {roc_auc2:.2f})', color=colors[1])
plt.plot(fpr3, tpr3, lw=2, label=f'DT ROC curve (area = {roc_auc3:.2f})', color=colors[2])
plt.plot(fpr4, tpr4, lw=2, label=f'RF ROC curve (area = {roc_auc4:.2f})', color=colors[3])
plt.plot(fpr5, tpr5, lw=2, label=f'xgboost ROC curve (area = {roc_auc5:.2f})', color=colors[4])
plt.plot(fpr6, tpr6, lw=2, label=f'lightgbm ROC curve (area = {roc_auc6:.2f})', color=colors[5])
plt.plot(fpr7, tpr7, lw=2, label=f'catboost ROC curve (area = {roc_auc7:.2f})', color=colors[6])
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=18)
plt.legend(loc='lower right')


plt.savefig(r'./images/2-Receiver Operating Characteristic (ROC) Curve.png', dpi=500)  

plt.show()

In [ ]:


from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


conf_matrix = confusion_matrix(y_test_vote, y_pred_vote)


classes = ['Non-MIBP','MIBP']
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes,annot_kws={"size": 14})
plt.xlabel('Predicted Labels', fontsize=15)
plt.ylabel('True Labels', fontsize=15)
plt.title('Confusion Matrix', fontsize=18)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)


plt.savefig(r'./images/2-Confusion Matrix.png', dpi=500) 
plt.show()


In [ ]:

from sklearn.metrics import precision_score, recall_score, f1_score


precision_vote = precision_score(y_test_vote, y_pred_vote)
recall_vote = recall_score(y_test_vote, y_pred_vote)
f1_vote = f1_score(y_test_vote, y_pred_vote)
print('mymodel')
print("Precision: {:.4f}".format(precision_vote))
print("Recall: {:.4f}".format(recall_vote))
print("F1 Score: {:.4f}".format(f1_vote))

precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
print('svm')
print("Precision: {:.4f}".format(precision_svm))
print("Recall: {:.4f}".format(recall_svm))
print("F1 Score: {:.4f}".format(f1_svm))


precision_DecisionTreeClassifier = precision_score(y_test, y_pred_DecisionTreeClassifier)
recall_DecisionTreeClassifier = recall_score(y_test, y_pred_DecisionTreeClassifier)
f1_DecisionTreeClassifier = f1_score(y_test, y_pred_DecisionTreeClassifier)
print('DecisionTreeClassifier')
print("Precision: {:.4f}".format(precision_DecisionTreeClassifier))
print("Recall: {:.4f}".format(recall_DecisionTreeClassifier))
print("F1 Score: {:.4f}".format(f1_DecisionTreeClassifier))


precision_RandomForestClassifier = precision_score(y_test, y_pred_RandomForestClassifier)
recall_RandomForestClassifier = recall_score(y_test, y_pred_RandomForestClassifier)
f1_RandomForestClassifier = f1_score(y_test, y_pred_RandomForestClassifier)
print('RandomForestClassifier')
print("Precision: {:.4f}".format(precision_RandomForestClassifier))
print("Recall: {:.4f}".format(recall_RandomForestClassifier))
print("F1 Score: {:.4f}".format(f1_RandomForestClassifier))

precision_xgboost = precision_score(y_test, y_pred_xgboost)
recall_xgboost = recall_score(y_test, y_pred_xgboost)
f1_xgboost = f1_score(y_test, y_pred_xgboost)
print('xgboost')
print("Precision: {:.4f}".format(precision_xgboost))
print("Recall: {:.4f}".format(recall_xgboost))
print("F1 Score: {:.4f}".format(f1_xgboost))


precision_lightgbm = precision_score(y_test, y_pred_lightgbm)
recall_lightgbm = recall_score(y_test, y_pred_lightgbm)
f1_lightgbm = f1_score(y_test, y_pred_lightgbm)
print('lightgbm')
print("Precision: {:.4f}".format(precision_lightgbm))
print("Recall: {:.4f}".format(recall_lightgbm))
print("F1 Score: {:.4f}".format(f1_lightgbm))


precision_catboost = precision_score(y_test, y_pred_catboost)
recall_catboost = recall_score(y_test, y_pred_catboost)
f1_catboost = f1_score(y_test, y_pred_catboost)
print('catboost')
print("Precision: {:.4f}".format(precision_catboost))
print("Recall: {:.4f}".format(recall_catboost))
print("F1 Score: {:.4f}".format(f1_catboost))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

categories = ['XLC-S-MIBP', 'SVM', 'DT', 'RF', 'xgboost', 'lightgbm', 'catboost']
num_categories = len(categories)
values_set1 = [accuracy_vote, accuracy_svm, accuracy_DecisionTreeClassifier, accuracy_RandomForestClassifier, 
               accuracy_xgboost, accuracy_lightgbm, accuracy_catboost]
values_set2 = [precision_vote, precision_svm, precision_DecisionTreeClassifier, precision_RandomForestClassifier, 
               precision_xgboost, precision_lightgbm, precision_catboost]
values_set3 = [recall_vote, recall_svm, recall_DecisionTreeClassifier, recall_RandomForestClassifier, 
               recall_xgboost, recall_lightgbm, recall_catboost]
values_set4 = [f1_vote, f1_svm, f1_DecisionTreeClassifier, f1_RandomForestClassifier, 
               f1_xgboost, f1_lightgbm, f1_catboost]

bar_width = 0.21
colors = sns.color_palette("colorblind", 4)

r = np.arange(num_categories)
r1 = r - 1.5*bar_width
r2 = r - 0.5*bar_width
r3 = r + 0.5*bar_width
r4 = r + 1.5*bar_width

plt.bar(r1, values_set1, color=colors[0], width=bar_width, edgecolor='grey', label='ACC')
plt.bar(r2, values_set2, color=colors[1], width=bar_width, edgecolor='grey', label='P')
plt.bar(r3, values_set3, color=colors[2], width=bar_width, edgecolor='grey', label='R')
plt.bar(r4, values_set4, color=colors[3], width=bar_width, edgecolor='grey', label='F1')

plt.xlabel('Categories', fontsize=15)
plt.ylabel('Values', fontsize=15)

plt.ylim(0.6, 1.0)

plt.xticks(r, categories, fontsize=8)  
plt.title('Model Evaluation', fontsize=18)

plt.legend(loc='upper right',fontsize=12)

plt.savefig(r'./images/2-Model Evaluation.png', dpi=500) 


plt.show()
